  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for panda: filename=panda-0.3.1-py3-none-any.whl size=7238 sha256=8b90c6cfc38109363762f3a939889182eb912b80120f8477d9fb5848f41d84b6
  Stored in directory: c:\users\roman\appdata\local\pip\cache\wheels\6c\5a\a9\a28c87238ddcc61701224f538bedccc21c9e2b8bc6fa8fca0e
Successfully built panda


In [4]:
import numpy as np
import pandas as pd

In [71]:
grid=np.array([
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,14,14,10,10,0,0,0],
    [0,0,0,0,0,0,27,27,0,0,0,0,0,0,0,0,0,0,0,14,14,14,14,10,10,10,10],
    [0,0,0,0,0,27,27,27,27,0,0,0,0,0,0,0,0,0,0,14,14,14,14,10,10,10,10],
    [0,0,0,27,27,27,27,27,27,27,0,0,0,0,0,0,0,14,14,14,14,14,14,17,17,10,10],
    [27,27,27,27,27,27,27,27,27,27,27,0,0,0,0,0,0,14,14,14,14,14,14,14,17,17,17],
    [27,27,27,27,27,27,27,27,27,27,5,5,14,14,14,14,14,14,14,14,2,2,18,18,18,0,0],
    [27,27,27,27,27,27,27,27,27,20,20,5,5,5,5,14,14,24,24,2,2,2,2,18,0,0,0],
    [0,27,27,27,27,26,26,26,20,20,20,20,20,20,5,5,19,21,24,24,22,22,22,22,0,0,0],
    [0,26,26,26,26,26,26,26,20,20,20,20,20,5,5,5,21,21,11,11,11,1,1,15,0,0,0],
    [0,26,26,26,26,26,26,26,26,26,20,20,20,20,5,21,21,25,12,12,11,16,1,15,0,0,0],
    [0,0,0,26,26,26,26,26,26,26,26,20,20,20,25,25,25,25,12,12,12,16,23,23,0,0,0],
    [0,0,0,0,26,26,26,26,26,26,26,25,25,25,25,3,3,8,13,13,16,16,4,23,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,28,28,25,25,25,8,8,8,13,13,13,4,4,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,28,28,25,28,28,28,7,7,7,7,6,6,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,28,28,28,28,7,7,6,6,6,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9,0,0,0,0,0,0,0,0,0,]
])
days=40
infected=pd.read_csv('data/time_series_covid19_confirmed_global.csv',usecols=range(4+days))
infected=infected[infected['Country/Region']=='China']
regions=pd.read_csv('data/regions.csv')
alpha,beta,gamma=0.21,1.4,1/5.1

In [72]:
ca_infected=[]
for i in range(days):
    aux=np.zeros((grid.shape[0],grid.shape[1]))
    for j in range(grid.shape[0]):
        for k in range(grid.shape[1]):
            if grid[j][k]!=0:
                region=regions[regions['ID']==grid[10,4]].values[0]
                aux[j][k]=(infected[infected['Province/State']==region[1]].values[0])[i+4]/region[2]
    ca_infected.append(aux)
delta_i=[]
p=[]
for i in range(1,len(ca_infected)):
    aux=ca_infected[i]-ca_infected[i-1]
    delta_i.append(aux)
for i in range(1,len(ca_infected)):
    aux=np.zeros((grid.shape[0],grid.shape[1],6))
    current_day=ca_infected[i-1]
    for j in range(grid.shape[0]):
        for k in range(grid.shape[1]):
            if j==0:
                if k==0:
                    current_cell=np.array([1,current_day[j][k],0,current_day[j][k+1],current_day[j+1][k],0])
                    aux[j][k]=np.concatenate((current_cell[:1],current_cell[1:]*(beta-gamma-alpha)))
                elif k==grid.shape[1]-1:
                    current_cell=np.array([1,current_day[j][k],0,0,current_day[j+1][k],current_day[j][k-1]])
                    aux[j][k]=np.concatenate((current_cell[:1],current_cell[1:]*(beta-gamma-alpha)))
                else:
                    current_cell=np.array([1,current_day[j][k],0,current_day[j][k+1],current_day[j+1][k],current_day[j][k-1]])
                    aux[j][k]=np.concatenate((current_cell[:1],current_cell[1:]*(beta-gamma-alpha)))
            elif j==grid.shape[0]-1:
                if k==0:
                    current_cell=np.array([1,current_day[j][k],current_day[j-1][k],current_day[j][k+1],0,0])
                    aux[j][k]=np.concatenate((current_cell[:1],current_cell[1:]*(beta-gamma-alpha)))
                elif k==grid.shape[1]-1:
                    current_cell=np.array([1,current_day[j][k],current_day[j-1][k],0,0,current_day[j][k-1]])
                    aux[j][k]=np.concatenate((current_cell[:1],current_cell[1:]*(beta-gamma-alpha)))
                else:
                    current_cell=np.array([1,current_day[j][k],current_day[j-1][k],current_day[j][k+1],0,current_day[j][k-1]])
                    aux[j][k]=np.concatenate((current_cell[:1],current_cell[1:]*(beta-gamma-alpha)))
            elif k==0:
                current_cell=np.array([1,current_day[j][k],current_day[j-1][k],current_day[j][k+1],current_day[j+1][k],0])
                aux[j][k]=np.concatenate((current_cell[:1],current_cell[1:]*(beta-gamma-alpha)))
            elif k==grid.shape[1]-1:
                current_cell=np.array([1,current_day[j][k],current_day[j-1][k],0,current_day[j+1][k],current_day[j][k-1]])
                aux[j][k]=np.concatenate((current_cell[:1],x[1:]*(beta-gamma-alpha)))
    p.append(aux)